In [99]:
import logging
import tweepy
import ast
import time 
import datetime as dt
import json
import dotenv
import os
import boto3
dotenv.load_dotenv()

BEARER_TOKEN = os.environ['BEARER_TOKEN']


rule_1 = "COVID19 lang:en"
rule_2 = "#covid19 lang:en"


def its_time(start_time, current_time, interval=20):
    stop = start_time + interval
    if current_time >= stop:
        return True
    return False

def send_message(msg):
    print(msg)
    #send_message.called = True
    

def package_message(raw):
    
    tweet_data = ast.literal_eval(raw.decode('UTF-8'))

    tweet_dict = dict(tweet=tweet_data['data']['text'])

    return tweet_data['data']['text']
    #json.dumps(tweet_dict, indent=2).encode('utf-8')

import cleantext
def clean_text(text):
    text = cleantext.remove_emoji(text)
    text = cleantext.replace_urls(text, '')
    text = text.replace('#', '').replace('RT', '').replace('\n', '').strip()

    return text

    


class MyStream(tweepy.StreamingClient):

    def on_connect(self):
        print("Connected to Stream!!!\n ===================")
        

    def on_data(self, raw_data):
        
        message = package_message(raw_data)

        message = clean_text(message)

        send_message(message)
       
        time.sleep(2)


    def on_error(self, status_code):
        if status_code == 420:
            # returning False in on_data disconnects the stream
            return False
    def on_errors(self, errors):
        
        error_msg = super().on_errors(errors)
        #notify(msg="ACTION REQUIRED: Error has occured while streaming from twitter.")

    def start_stream(self):
        self.filter(tweet_fields=['referenced_tweets'])

    def on_connection_error(self):
        print("Could not connect to the Stream")
        return super().on_connection_error()
    
    def on_disconnect(self):
        raise Exception

def reset_rules(stream):
    rules = stream.get_rules().data
    rules_ids =[]
    for rule in rules:
        rules_ids.append(rule.id)

    stream.delete_rules(rules_ids)
    stream.add_rules(tweepy.StreamRule(rule_1, tag="covid"))
    stream.add_rules(tweepy.StreamRule(rule_2, tag="covid"))


In [100]:
stream = MyStream(BEARER_TOKEN)
reset_rules(stream)
stream.start_stream()

Connected to Stream!!!
@WHO Utter and complete nonsense. Unscientific and not at all in line with the evidence. Masks are social control and extremely unhealthy to all people for multiple reasons.
@joyhealthcare01 How and why does kale hurt your thyroid?
@WHO That's why people were still getting sick despite wearing masks.  Just let it go already
@EvanBlake17: The CDC's cover-up of data on XBB15 is among the greatest scandals of the COVID19 pandemic, but the corporate media rema…
Just mask  up folks.Let's get this done without drama...just help each other out.
@LetsGoBrando45: Refusing the Covid19 Vaccine was justified at the onset of the ‘pandemic’ due to Pfizer’s extensive history of crimi…
@WHO The evidence that it “saves lives” is exceedingly poor. I will continue not wearing one, despite the mRNA-poisoned people spewing their immune-deficient covid gunk all over the place around me.
@SRAnesthesiaICU: A very nice editorial on our systematic review on barotrauma (16%) pneumothorax (

Exception: 

In [102]:
from bs4 import BeautifulSoup
import requests

In [181]:
url = """https://www.worldometers.info/coronavirus/"""

def scrape(url):
    info = {}
    res = requests.get(url=url)

    if res.status_code != 200:
        print(f"Access Denied!!\n Erro {res.status_code}")

    else:
        soup = BeautifulSoup(res.content)  
        data = soup.select('body')[0]
        content = data.select('div.container')[1].select('div.row')[1].select('div.col-md-8')[0].select('div.content-inner')[0].\
        select('div')

        # Get time number of cases was updated 
        update_time = content[1].text.strip()

        # Get the number of cases
        number_cases = content[3].select('div.maincounter-number')[0].select('span')[0].text.strip()
        #number_cases = number_cases.replace(',','')

        info = dict(time = update_time, cases = number_cases)

        return info

In [179]:
data = scrape(url)

In [180]:
data

{'time': 'Last updated: January 06, 2023, 12:04 GMT', 'cases': '667,506,481'}